Implement a optimization method for feature combination / selection 

In [6]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

vhr.plot.VisualizeParams.renderer = 'vscode' 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# -- LOAD A DATASET

dataset_name = 'mr_nirp'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames
print("Number of videos: ", len(allvideo))  

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

MOTION = [0,2,4,6,9,11,13]
STILL = [1,3,5,7,8,10,12]

Number of videos:  15
0 D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
2 D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\RGB_corrected\Subject2_motion_940.avi
3 D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\RGB_corrected\Subject2_still_940.avi
4 D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\RGB_corrected\Subject3_motion_940.avi
5 D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\RGB_corrected\Subject3_still_940.avi
6 D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\RGB_corrected\Subject4_motion_940.avi
7 D:/datasets_rppg/MR-NIRP_indoor\Subject4_still_940-004\Subject4_still_940\RGB_corrected\Subject4_still_940.avi
8 D:/datasets_rppg/MR-NIRP_indoor\Subject5_still_940-003\Subject5_still_940\RG

In [8]:
# -- PARAMETER SETTING

roi = 'mustache'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 2     # minimum number of landmarks tested
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for mr_nirp is 60 
Testing ROI = mustache with minimum 2 landmarks


# Hyperopt

https://celmore25.medium.com/automated-feature-selection-with-hyperopt-46865e1b4fce

Documentation: https://hyperopt.github.io/hyperopt/#documentation

Code: https://github.com/hyperopt/hyperopt

- It does not look like the fmin works too well when the numbe rof max iterations is less than the numbre of combinations. looks like ill have to do an exaustive search after all. 
- I can discard lips  and temple area

TODO change from run mlti methods to run dataset ? 


In [9]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [10]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, hp, space_eval, FMinIter
from hyperopt_edit import refresh, run
from hyperopt.base import JOB_STATE_DONE
from hyperopt.early_stop import no_progress_loss
from itertools import chain, combinations

def get_combinations(elements, min_len=2):
    combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len, len(elements) + 1)))
    return [list(i) for i in combs]

# parameter space
parameter_space = {}
ldmks_list = rois[roi]

# ldmks_list = ['chin', 'glabella']
for ldmk in ldmks_list:
    parameter_space[ldmk] = hp.choice(ldmk, [0, 1])
print(f"Selection roi as {roi} with landmarks {ldmks_list}")
print(f"Number of possible combinations with {len(ldmks_list)} landmarks is :", 2**len(ldmks_list) - 1)

all_combinations = get_combinations(parameter_space.keys(), min_len)
for elem in all_combinations:
    elem.sort()
print(f"For min length {min_len}, number of all combinations is {len(all_combinations)}")
print(all_combinations)

valid_combinations = []
for comb in all_combinations:
    valid = True
    if 'left' in ''.join(comb):
        lefts = [s for s in comb if 'left' in s]
        for elem in lefts:
            if '_'.join(['right']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if 'right' in ''.join(comb):
        rights = [s for s in comb if 'right' in s]
        for elem in rights:
            if '_'.join(['left']+elem.split('_')[1:]) not in comb:
                valid = False
                break
    if valid:
        valid_combinations.append(comb)
print(f"For min length {min_len}, number of valid combinations is {len(valid_combinations)}")
print(valid_combinations)

Selection roi as mustache with landmarks ['left_nasolabial_fold', 'right_nasolabial_fold', 'left_upper_lip', 'right_upper_lip', 'philtrum']
Number of possible combinations with 5 landmarks is : 31
For min length 2, number of all combinations is 26
[['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_nasolabial_fold', 'left_upper_lip'], ['left_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum'], ['left_upper_lip', 'right_nasolabial_fold'], ['right_nasolabial_fold', 'right_upper_lip'], ['philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_upper_lip', 'philtrum'], ['philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold'], ['left_nasolabial_fold', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_nasolabial_fold', 'left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum'], ['left_nasolabial_

In [11]:
from hyperopt.early_stop import no_progress_loss

class HpOptBinarySelect:
    def __init__(self, videos, dataset, winsize, patch_size, space, min_len, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.parameter_space = space
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.all_combinations = get_combinations(space.keys(), min_len)
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.load_ground_truth(videos)

    def get_combinations(elements, min_len=2):
        combs = list(chain.from_iterable(combinations(elements, r) for r in range(min_len, len(elements) + 1)))
        return [list(i) for i in combs]

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def objective(self,params):
        ldmks_list = [i for i,j in params.items() if j==1]
        print(f"LANDMARKS: {ldmks_list}")

        # TODO maybe i should just define params as such
        if ldmks_list not in valid_combinations:
            return {'loss': np.nan,'status': STATUS_FAIL}
        self.all_combinations = valid_combinations

        # Less than 2 landmarks tested
        if len(ldmks_list) < min([len(combs) for combs in self.all_combinations]):
            return {'loss': np.nan,'status': STATUS_FAIL}
    
        # Get past landmarks that were tested
        past_param_values = [trial['misc']['vals'] for trial in self.trials._dynamic_trials if trial['result']['status'] == 'ok']
        past_landmarks = []
        for value in past_param_values:
            past_landmarks.append([i for i,j in value.items() if j[0]==1])

        # All combinations have been tested
        if len(past_param_values) == len(self.all_combinations):
            print(f"All {len(past_param_values)} combinations have been tested ")
            self.trials._fmin_cancelled = True
            return {'loss': np.nan,'status': STATUS_FAIL}

        # Remove duplicate trials
        if ldmks_list in past_landmarks:
            return {'loss': np.nan,'status': STATUS_FAIL}
        
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            print(videoName)
            res = self.pipeline.run_on_video_multimethods(
                    ldmks_list=ldmks_list, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], 
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        print(f"Total loss for {ldmks_list}: {sum(losses)}")
        return {'loss': sum(losses), 'status': STATUS_OK}

    def optimize(self, max_evals=20):
        self.trials = Trials()
        best = fmin(fn=self.objective,
                    space=self.parameter_space,
                    algo=tpe.suggest,
                    max_evals=max_evals,
                    trials=self.trials,
                    # early_stop_fn=no_progress_loss(10),
                    )
        return space_eval(self.parameter_space, best)
    
FMinIter.run = run
Trials.refresh = refresh

In [13]:
# np.arange(0, len(dataset.videoFilenames))
pl = custom_pipeline.LandmarksPipeline()
hpobj = HpOptBinarySelect(
    videos=np.arange(0, len(dataset.videoFilenames)), dataset=dataset, winsize=8, patch_size=patch_size, 
    space=parameter_space, min_len=2, pipeline=pl, methods=['cupy_CHROM'], verb=False
)

best = hpobj.optimize(max_evals=30)
out = [i for i,j in best.items() if j==1]
print(f'Final number of features {len(out)}: ', out)

Video name:  {15} ['D:/datasets_rppg/MR-NIRP_indoor\\Subject1_motion_940\\Subject1_motion_940\\RGB_corrected\\Subject1_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject1_still_940-015\\Subject1_still_940\\RGB_corrected\\Subject1_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject2_motion_940\\Subject2_motion_940\\RGB_corrected\\Subject2_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject2_still_940-002\\Subject2_still_940\\RGB_corrected\\Subject2_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject3_motion_940\\Subject3_motion_940\\RGB_corrected\\Subject3_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject3_still_940-012\\Subject3_still_940\\RGB_corrected\\Subject3_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject4_motion_940\\Subject4_motion_940\\RGB_corrected\\Subject4_motion_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject4_still_940-004\\Subject4_still_940\\RGB_corrected\\Subject4_still_940.avi', 'D:/datasets_rppg/MR-NIRP_indoor\\Subject

In [13]:
df = hpobj.res.reset_index(drop=True)
if dataset_name == 'lgi_ppgi':
  df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
if dataset_name == 'mr_nirp':
  df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
indexes = {}
for v in range(len(allvideo)):
  indexes[allvideo[v].split('\\')[2]] = v
indexes = pd.DataFrame({'videoIdx':list(indexes.values()), 'videoFilename':list(indexes.keys())})
df = df.drop(columns=['videoIdx'])
df.insert(3, 'videoIdx', df['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

from fastdtw import fastdtw
df.insert(11, 'DTW', None)
for row in df.itertuples():
  distance, path = fastdtw(row.bpmGT, row.bpmES)
  df.loc[row.Index, 'DTW'] = [distance]

print("Number of unique combinations tested: ", df.landmarks.apply(lambda x: tuple(x)).unique().size)
print("Best score achieved with : ", out)
print(f"Number of NaN rows {df['bpmGT'].isna().sum()} ({df[df['bpmGT'].isna()]['videoFilename'].iloc[0] if df['bpmGT'].isna().sum().sum() > 0 else None})")
df[df['landmarks'].apply(lambda x: x == out)][['videoFilename', 'RMSE' ,'MAE', 'PCC']]

# df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
# df['PCC'] = df['PCC'].apply(lambda x: abs(x[0])) 
# df[['landmarks', 'videoIdx', 'MAE', 'RMSE', 'PCC']].groupby('landmarks').mean().sort_values(by='RMSE')

Number of unique combinations tested:  22


NameError: name 'out' is not defined

In [9]:
df.head(1)

,method,dataset,sigFilename,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,DTW,MAD,bpmGT,bpmES,bpmES_mad,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,,,0,Subject1_motion_940,[2.0402644633246516],[1.1526131465517242],[0.7880692591861292],[0.7760439490403377],[-0.38217448768936546],[9.51953125],[68.873046875],"[1.7578125, 1.318359375, 1.7578125, 1.31835937...","[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[71.630859375, 75.5859375, 77.34375, 79.101562...",,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",47.487194,"[glabella, left_lower_lateral_forehead, right_..."


In [14]:
df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
df = df.drop(columns=['sigFilename', 'bpmES_mad'])
df['dataset'] = f'{dataset_name}_{roi}'

print("saving into ", f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5')
df.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')
# df_old = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
# df_new = pd.concat([df_old, df], ignore_index=True)
# df_new.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')

saving into  ../results/test_landmarks/MR_NIRP_nose.h5


C:\Users\erolland\AppData\Local\Temp\3\ipykernel_24472\3481480161.py:6: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'DTW', 'MAD', 'bpmGT', 'bpmES', 'timeGT', 'timeES',
       'landmarks'],
      dtype='object')]

  df.to_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df', mode='w')


In [25]:
x = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df')
# x = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
print(x.videoIdx.unique())
print(x.landmarks.unique().size)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
3


In [26]:
x.head(2)

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,DTW,MAD,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,cupy_CHROM,mr_nirp_cheeks,0,Subject1_motion_940,[1.7726826470146042],[0.9025794719827587],[0.8488353163618159],[0.8229857128110615],[1.314836561409877],[8.640625],[56.958984375],"[1.318359375, 0.87890625, 0.87890625, 0.878906...","[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[69.873046875, 75.5859375, 78.22265625, 79.101...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",49.667503,"(left_malar, right_malar)"
1,cupy_CHROM,mr_nirp_cheeks,1,Subject1_still_940,[1.3236925129284862],[0.801280886627907],[0.8588910595787541],[0.8541416375044004],[1.669057860082492],[8.23828125],[107.91796875],"[1.7578125, 1.7578125, 1.318359375, 1.31835937...","[76.0, 76.5, 77.0, 76.5, 76.0, 76.0, 76.0, 76....","[76.46484375, 76.46484375, 75.5859375, 74.7070...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",134.386794,"(left_malar, right_malar)"


In [24]:
x[['landmarks', 'videoIdx','videoFilename', 'MAE', 'RMSE', 'PCC']].sort_values(['landmarks', 'videoIdx'])

,landmarks,videoIdx,videoFilename,MAE,RMSE,PCC
60,"(glabella, left_lower_lateral_forehead, lower_...",0,Subject1_motion_940,[1.1018318965517242],[1.9446997697772423],[0.8079255761454256]
61,"(glabella, left_lower_lateral_forehead, lower_...",1,Subject1_still_940,[0.8212209302325582],[1.2986728970997385],[0.8647586170070111]
62,"(glabella, left_lower_lateral_forehead, lower_...",2,Subject2_motion_940,[0.6614829009433962],[0.8598895931972573],[0.9805386941512182]
63,"(glabella, left_lower_lateral_forehead, lower_...",3,Subject2_still_940,[63.68134236453202],[67.26737004414912],[0.1642633082725681]
64,"(glabella, left_lower_lateral_forehead, lower_...",4,Subject3_motion_940,[0.5341981132075472],[0.694197368053355],[0.9742522078427989]
...,...,...,...,...,...,...
25,"(left_lower_lateral_forehead, right_lower_late...",10,Subject6_still_940,[2.5029611895161286],[3.5822613985577343],[0.29349082067099364]
26,"(left_lower_lateral_forehead, right_lower_late...",11,Subject7_motion_940,[1.4626326650943395],[2.313325616886147],[0.8528939973098923]
27,"(left_lower_lateral_forehead, right_lower_late...",12,Subject7_still_940,[0.993508399566474],[1.64424438916093],[0.9507921124656264]
28,"(left_lower_lateral_forehead, right_lower_late...",13,Subject8_motion_940,[1.9279186320754718],[5.400910058366144],[0.599633410423806]


# AAAH FIX

In [100]:
# x = pd.read_hdf(f'../results/test_landmarks/{dataset_name.upper()}_{roi}.h5', key='df')
x = pd.read_hdf(f'../results/test_landmarks/h5/{dataset_name.upper()}/{dataset_name.upper()}_{roi}.h5', key='df')
x = x.sort_values("videoFilename").reset_index(drop=True)
print(x.shape)
print(len(x.videoIdx.unique()), x.videoIdx.unique())
print(x.landmarks.unique().size)

(45, 16)
15 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
3


In [102]:
sigGT_list = []
bpmGT_list = []
timesGT_list = []
videoFileName = []
winsize = 8
videos = np.arange(0, len(dataset.videoFilenames))

for videoIdx in videos:
    fname = dataset.getSigFilename(videoIdx)
    print(fname)
    sigGT = dataset.readSigfile(fname)
    bpmGT, timesGT = sigGT.getBPM(winsize)
    sigGT_list.append(sigGT)
    bpmGT_list.append(bpmGT)
    timesGT_list.append(timesGT)
    videoFileName.append(dataset.getVideoFilename(videoIdx))
    fps = vhr.extraction.get_fps(videoFileName[-1]) # assuming they are all the same
print('Video name: ', {len(videoFileName)}, videoFileName)
print('Video frame rate: ',fps)

D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject4_still_940-004\Subject4_still_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject5_still_940-003\Subject5_still_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject6_motion_940-008\Subject6_motion_940\PulseOX\pulseOx.mat
D:/datasets_rppg/MR-NIRP_indoor\Subject6_still_940-005\Subject6_st

In [103]:
x.head(2)

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,bpmGT,bpmES,timeGT,timeES,landmarks
0,cupy_CHROM,mr_nirp_jaw,0,Subject1_motion_940,[38.0852568082684],[34.08075161637931],[0.22554272337206172],[0.030457962972525957],[-3.9529915256746886],[83.126953125],"[34.27734375, 30.322265625, 30.322265625, 25.4...","[63.0, 66.5, 73.0, 74.5, 75.0, 74.5, 73.5, 72....","[95.361328125, 97.119140625, 115.13671875, 94....","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(left_marionette_fold, right_marionette_fold)"
1,cupy_CHROM,mr_nirp_jaw,0,Subject1_motion_940,[33.21428405854258],[30.324185075431036],[0.2400727042392256],[0.0328663120710825],[-3.9100219155180045],[58.95703125],"[36.03515625, 32.080078125, 29.443359375, 26.3...","[63.0, 66.5, 73.0, 74.5, 75.0, 74.5, 73.5, 72....","[92.724609375, 89.208984375, 103.7109375, 87.4...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....","(chin, left_marionette_fold, right_marionette_..."


In [104]:
bpmES = list(x.bpmES)
timesES = list(x.timeES)
nb = x.landmarks.unique().size # nb landmarks
print(nb)
print(len(bpmES), len(bpmGT_list), len(timesES), len(timesGT_list))
for i in range(len(bpmES)):
    print(x.videoFilename[i])
    RMSE, MAE, MAX, PCC, CCC, SNR = getErrors(None, fps, bpmES[i], bpmGT_list[i//nb], timesES[i], timesGT_list[i//nb])
    x.loc[i,'RMSE'] = RMSE
    x.loc[i,'MAE'] = MAE
    x.loc[i,'MAX'] = MAX
    x.loc[i,'PCC'] = PCC
    x.loc[i,'CCC'] = CCC
    x.loc[i,'SNR'] = SNR
    x.at[i,'bpmGT'] = bpmGT_list[i//nb]
    x.at[i,'timeGT'] = timesGT_list[i//nb]   

3
45 15 45 15
Subject1_motion_940
Subject1_motion_940
Subject1_motion_940
Subject1_still_940
Subject1_still_940
Subject1_still_940
Subject2_motion_940
Subject2_motion_940
Subject2_motion_940
Subject2_still_940
Subject2_still_940
Subject2_still_940
Subject3_motion_940
Subject3_motion_940
Subject3_motion_940
Subject3_still_940
Subject3_still_940
Subject3_still_940
Subject4_motion_940
Subject4_motion_940
Subject4_motion_940
Subject4_still_940
Subject4_still_940
Subject4_still_940
Subject5_still_940
Subject5_still_940
Subject5_still_940
Subject6_motion_940
Subject6_motion_940
Subject6_motion_940
Subject6_still_940
Subject6_still_940
Subject6_still_940
Subject7_motion_940
Subject7_motion_940
Subject7_motion_940
Subject7_still_940
Subject7_still_940
Subject7_still_940
Subject8_motion_940
Subject8_motion_940
Subject8_motion_940
Subject8_still_940
Subject8_still_940
Subject8_still_940


In [37]:
MAX

array([77.62695312])

# Test Hyperopt

In [15]:
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials, hp, space_eval, FMinIter
from hyperopt_edit import refresh, run

FMinIter.run = run
Trials.refresh = refresh
trials = Trials()

def func(x):
    print(f"RUN {x}")
    past_param_values = [trial['misc']['vals'] for trial in trials._dynamic_trials if trial['result']['status'] == 'ok']
    print(f"Past landmarks: {past_param_values}")
    if len(past_param_values) >= 3:
        print(f"All combinations have been tested {len(past_param_values)}")
        print(f"first one should be safe {trials.trials[0]} ")
        if hasattr(trials.trials, "_fmin_cancelled"):
            print(f"Cancelling trials {trials._fmin_cancelled}")
        trials._fmin_cancelled = True
        return {"loss":np.nan,"status": STATUS_FAIL}
    
    if (x < 0):
        return {"loss": (x - 3) ** 2, "status": STATUS_OK}  
    else:
        print(f"ERROR ERROR ERROR {len(trials._dynamic_trials)}")
        return {"loss":np.nan,"status": STATUS_FAIL}

argmin = fmin(
    fn=func,
    space=hp.uniform("x", -1, 2),
    algo=tpe.suggest,
    max_evals=5,
    trials=trials,
    catch_eval_exceptions = False,
)

print(argmin)

                                                     
     while 2: True, False
     while 3 qlen, self.max_queue_len, n_queued, N: 0, 1, 0, 5, True, True, True
     n_queued: 0 q_len 1                             
RUN 1.5087915254710804                               
Past landmarks: []                                   
ERROR ERROR ERROR 1                                  
     while 2.1 unfinished, done, done this iteration: 0 0 0
     while 2.2: 0 5, True False False                
                                                     
     while 2: True, False
     while 3 qlen, self.max_queue_len, n_queued, N: 0, 1, 0, 5, True, True, True
     n_queued: 0 q_len 1                             
RUN 1.809393828244759                                
Past landmarks: []                                   
ERROR ERROR ERROR 2                                  
     while 2.1 unfinished, done, done this iteration: 0 0 0
     while 2.2: 0 5, True False False                
                  

In [16]:
print("Number of trials: ", len(trials._dynamic_trials), len(trials.trials))

Number of trials:  8 3
